# Linear Regression using Pyspark

In [1]:
#create sparksession object
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('lin_reg').getOrCreate()

In [2]:
#import Linear Regression from spark's MLlib
from pyspark.ml.regression import LinearRegression

In [3]:
#Load the dataset
df=spark.read.csv('Linear_regression_dataset.csv',inferSchema=True,header=True)

In [4]:
#validate the size of data
print((df.count(), len(df.columns)))

(1232, 6)


In [5]:
#explore the data
df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)



In [6]:
#view statistical measures of data 
df.describe().show(3,False)

+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|var_1            |var_2            |var_3             |var_4               |var_5               |output             |
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|count  |1232             |1232             |1232              |1232                |1232                |1232               |
|mean   |715.0819805194806|715.0819805194806|80.90422077922078 |0.3263311688311693  |0.25927272727272715 |0.39734172077922014|
|stddev |91.5342940441652 |93.07993263118064|11.458139049993724|0.015012772334166148|0.012907228928000298|0.03326689862173776|
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
only showing top 3 rows



In [7]:
#sneak into the dataset
df.head(3)

[Row(var_1=734, var_2=688, var_3=81, var_4=0.328, var_5=0.259, output=0.418),
 Row(var_1=700, var_2=600, var_3=94, var_4=0.32, var_5=0.247, output=0.389),
 Row(var_1=712, var_2=705, var_3=93, var_4=0.311, var_5=0.247, output=0.417)]

In [8]:
#import corr function from pyspark functions
from pyspark.sql.functions import corr

In [47]:
# check for correlation
df.select(corr('var_1','output')).show()

+-------------------+
|corr(var_1, output)|
+-------------------+
| 0.9187399607627283|
+-------------------+



In [10]:
#import vectorassembler to create dense vectors
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [11]:
#select the columns to create input vector
df.columns

['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'output']

In [12]:
#create the vector assembler 
vec_assmebler=VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'],outputCol='features')

In [13]:
#transform the values
features_df=vec_assmebler.transform(df)

In [14]:
#validate the presence of dense vectors 
features_df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
#view the details of dense vector
features_df.select('features').show(1,False)

+------------------------------+
|features                      |
+------------------------------+
|[734.0,688.0,81.0,0.328,0.259]|
+------------------------------+
only showing top 1 row



In [16]:
#create data containing input features and output column
model_df=features_df.select('features','output')

In [17]:
model_df.show(3)

+--------------------+------+
|            features|output|
+--------------------+------+
|[734.0,688.0,81.0...| 0.418|
|[700.0,600.0,94.0...| 0.389|
|[712.0,705.0,93.0...| 0.417|
+--------------------+------+
only showing top 3 rows



In [18]:
#size of model df
print((model_df.count(), len(model_df.columns)))

(1232, 2)


### Split Data - Train & Test sets


In [19]:
#split the data into 70/30 ratio for train test purpose
train_df,test_df=model_df.randomSplit([0.7,0.3])

In [20]:
print((train_df.count(), len(train_df.columns)))

(882, 2)


In [21]:
print((test_df.count(), len(test_df.columns)))

(350, 2)


In [52]:
train_df.describe().show()

+-------+-------------------+
|summary|             output|
+-------+-------------------+
|  count|                871|
|   mean| 0.3967841561423649|
| stddev|0.03300416938751066|
|    min|              0.301|
|    max|              0.485|
+-------+-------------------+



## Build Linear Regression Model 

In [22]:
#Build Linear Regression model 
lin_Reg=LinearRegression(labelCol='output')

In [23]:
#fit the linear regression model on training data set 
lr_model=lin_Reg.fit(train_df)

In [48]:
lr_model.intercept

0.20596014754214345

In [26]:
    print(lr_model.coefficients)

[0.000345569740987,6.07805293067e-05,0.000269273376209,-0.713663600176,0.432967466411]


In [32]:
training_predictions=lr_model.evaluate(train_df)

In [37]:
training_predictions.meanSquaredError

0.0001488484750026884

In [33]:
training_predictions.r2

0.8656062610679494

In [94]:
#make predictions on test data 
test_results=lr_model.evaluate(test_df)

In [95]:
#view the residual errors based on predictions 
test_results.residuals.show(10)

+--------------------+
|           residuals|
+--------------------+
|0.009931058530749659|
|-0.00139303903577...|
|-0.01193666754368...|
|-0.00477589231673...|
|-0.01348565590770...|
|0.007370771757058858|
|0.011656259678168412|
|0.002023276514917...|
|-0.00148012921050...|
|-0.01349400001445...|
+--------------------+
only showing top 10 rows



In [96]:
#coefficient of determination value for model
test_results.r2

0.8716898064262081

In [97]:
test_results.rootMeanSquaredError

0.012126612208688329

In [98]:
test_results.meanSquaredError

0.00014705472365990883